In [1]:
!pip install fastai==0.7.0

  Using cached https://files.pythonhosted.org/packages/50/6d/9d0d6e17a78b0598d5e8c49a0d03ffc7ff265ae62eca3e2345fab14edb9b/fastai-0.7.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6b/00/f7b4d7fd901db9a79d63e88000bd1e12efba4f5fb52608f906d7fea2b18f/pandas_summary-0.0.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1f/48/4e1461d828baf41d609efaa720d20090ac6ec346b5daad3c88e243e2207e/sklearn_pandas-1.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/08/55/940b97cc6f19a19f5dab9efef2f68a0ce43a7632f858b272391f0b851a7e/feather-format-0.4.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/c2/d4/fe7e2637975c476734fcbf53776e650a29680194eb0dd21dbdc020ca92de/isoweek-1.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8d/ff/13e77ee7ac431f831e20d81a6bf0214ca1cf550cf9b575e3213e14325c81/opencv_python-4.1.0.25-cp37-cp37m-macosx_10_7_x86_64.macosx_10_9_intel.macosx_10_9_x86_64.m

In [2]:
# Import
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# from fastai.structured import add_datepart

# Linear Regression
from sklearn.linear_model import LinearRegression

# k-Nearest Neighbors
from sklearn import neighbors
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

# Long-term Memory
from sklearn.preprocessing import MinMaxScaler
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, LSTM, Activation

plt.figure(figsize=(16,8))

<Figure size 1600x800 with 0 Axes>

# Data Collecting

In [3]:
# Config
symbl = 'MSFT'
apiKey = 'MSXSR0YHC991CZN6'

# Fetch Historical Data
df = pd.read_csv('https://www.alphavantage.co/query?datatype=csv&function=TIME_SERIES_MONTHLY&symbol=' + symbl + '&outputsize=compact&apikey=' + apiKey)

# Setting index as date
df['timestamp'] = pd.to_datetime(df.timestamp, format='%Y-%m-%d')
df.index = df['timestamp']

# Data Overview

In [4]:
# Get general statistics
df.describe()

,open,high,low,close,volume
count,256.000000,256.000000,256.000000,256.000000,2.560000e+02
mean,49.753136,53.349432,46.191372,49.663046,9.898068e+08
std,29.927316,33.005746,26.664632,29.707028,4.365882e+08
min,16.035000,18.880000,14.870000,16.150000,2.155232e+08
25%,27.235000,28.215000,25.855000,27.117425,6.457838e+08
50%,36.160000,37.695000,32.740000,36.205000,9.474929e+08
75%,64.358750,69.947500,60.617500,64.320000,1.301201e+09
max,175.440000,180.380000,143.980000,175.000000,3.044579e+09


In [5]:
# Get data types
df.dtypes

timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume                int64
dtype: object

In [15]:
# Check top 10 rows
df.head(10)

,timestamp,open,high,low,close,volume
timestamp,,,,,,
2019-05-10,2019-05-10,130.530,130.650,123.5700,127.13,225910019
2019-04-30,2019-04-30,118.950,131.370,118.1000,130.60,433157868
2019-03-29,2019-03-29,112.890,120.820,108.8000,117.94,589045341
2019-02-28,2019-02-28,103.775,113.240,102.3500,112.03,469095970
2019-01-31,2019-01-31,99.550,107.900,97.2000,104.43,714204787
2018-12-31,2018-12-31,113.000,113.420,93.9600,101.57,944287635
2018-11-30,2018-11-30,107.050,112.240,99.3528,110.89,720228643
2018-10-31,2018-10-31,114.750,116.180,100.1100,106.81,927547942
2018-09-28,2018-09-28,110.850,115.290,107.2300,114.37,480255674


In [16]:
# Check last 10 rows
df.tail(10)

,timestamp,open,high,low,close,volume
timestamp,,,,,,
1998-11-30,1998-11-30,106.37,130.27,104.75,122.00,290676700
1998-10-30,1998-10-30,108.06,110.12,87.75,105.87,410762300
1998-09-30,1998-09-30,95.25,114.62,94.50,110.06,298081100
1998-08-31,1998-08-31,109.12,113.75,95.75,95.94,337509900
1998-07-31,1998-07-31,109.12,119.62,105.37,109.94,307324500
1998-06-30,1998-06-30,83.87,108.56,83.12,108.37,308288100
1998-05-29,1998-05-29,90.19,91.00,81.87,84.81,273845600
1998-04-30,1998-04-30,89.81,99.12,86.62,90.12,254744300
1998-03-31,1998-03-31,85.87,90.94,79.25,89.50,274981900


## Prepare data to merge with sentimental analysis

In [6]:
cropped_df = df[(df['timestamp'].dt.year >= 2000)]

In [7]:
cropped_df.tail(10)

,timestamp,open,high,low,close,volume
timestamp,,,,,,
2000-10-31,2000-10-31,60.50,70.12,48.44,68.87,1234707800
2000-09-29,2000-09-29,70.00,72.06,58.63,60.31,712766900
2000-08-31,2000-08-31,69.94,74.87,68.12,69.81,609699900
2000-07-31,2000-07-31,79.69,82.87,67.25,69.81,617092900
2000-06-30,2000-06-30,64.37,82.19,63.81,80.00,733525100
2000-05-31,2000-05-31,72.87,74.00,60.38,62.56,672215400
2000-04-28,2000-04-28,94.44,96.50,65.00,69.75,1129073300
2000-03-31,2000-03-31,89.62,115.00,88.94,106.25,1014093800
2000-02-29,2000-02-29,98.50,110.00,88.12,89.37,667243800


In [8]:
cropped_df = cropped_df.reset_index(drop=True)

In [34]:
news_data = pd.read_csv("data/sentimental_data.csv")

In [10]:
news_data.head(10)

,year,month,positive
0,2000,1,0.541624
1,2000,2,0.537205
2,2000,3,0.537788
3,2000,4,0.536477
4,2000,5,0.541516
5,2000,6,0.532644
6,2000,7,0.537956
7,2000,8,0.539437
8,2000,9,0.535648
9,2000,10,0.534441


In [46]:
news_data.tail(10)

,year,month,positive
223,2018,8,0.565977
224,2018,9,0.553506
225,2018,10,0.548968
226,2018,11,0.552921
227,2018,12,0.561755
228,2019,1,0.548660
229,2019,2,0.556610
230,2019,3,0.562695
231,2019,4,0.544137
232,2019,5,0.543147


In [35]:
news_data = news_data[::-1].reset_index().drop(columns=['index'])

In [36]:
news_data.head(10)

,year,month,positive
0,2019,5,0.543147
1,2019,4,0.544137
2,2019,3,0.562695
3,2019,2,0.556610
4,2019,1,0.548660
5,2018,12,0.561755
6,2018,11,0.552921
7,2018,10,0.548968
8,2018,9,0.553506
9,2018,8,0.565977


In [37]:
sentimental_df = pd.concat([cropped_df, news_data], axis=1, join_axes=[cropped_df.index])

In [38]:
sentimental_df.tail(10)

,timestamp,open,high,low,close,volume,year,month,positive
223,2000-10-31,60.50,70.12,48.44,68.87,1234707800,2000,10,0.534441
224,2000-09-29,70.00,72.06,58.63,60.31,712766900,2000,9,0.535648
225,2000-08-31,69.94,74.87,68.12,69.81,609699900,2000,8,0.539437
226,2000-07-31,79.69,82.87,67.25,69.81,617092900,2000,7,0.537956
227,2000-06-30,64.37,82.19,63.81,80.00,733525100,2000,6,0.532644
228,2000-05-31,72.87,74.00,60.38,62.56,672215400,2000,5,0.541516
229,2000-04-28,94.44,96.50,65.00,69.75,1129073300,2000,4,0.536477
230,2000-03-31,89.62,115.00,88.94,106.25,1014093800,2000,3,0.537788
231,2000-02-29,98.50,110.00,88.12,89.37,667243800,2000,2,0.537205
232,2000-01-31,117.37,118.62,94.87,97.87,637437600,2000,1,0.541624
